In [1]:
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np
from pathlib import Path
from stitching.images import Images
from stitching.feature_detector import FeatureDetector
from stitching.feature_matcher import FeatureMatcher
from stitching.subsetter import Subsetter
from stitching.camera_estimator import CameraEstimator
from stitching.camera_adjuster import CameraAdjuster
from stitching.camera_wave_corrector import WaveCorrector
from stitching.warper import Warper
from stitching.timelapser import Timelapser
from stitching.cropper import Cropper
from stitching.seam_finder import SeamFinder
from stitching.exposure_error_compensator import ExposureErrorCompensator
from stitching.blender import Blender
from stitching import Stitcher
from stitching import AffineStitcher

In [ ]:
# allow displaying resulting images within notebook and load the correct img paths to used image sets

def plot_image(img, figsize_in_inches=(5,5)):
    fig, ax = plt.subplots(figsize=figsize_in_inches)
    ax.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    plt.show()
    
def plot_images(imgs, figsize_in_inches=(5,5)):
    fig, axs = plt.subplots(1, len(imgs), figsize=figsize_in_inches)
    for col, img in enumerate(imgs):
        axs[col].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    plt.show()

def get_image_paths(img_set):
    return [str(path.relative_to('.')) for path in Path('images').rglob(f'{img_set}*')]

imgs = get_image_paths("IMG")

## Resize Images

The first step is to resize the images to medium resolution.

In [ ]:
images = Images.of(imgs)
medium_imgs = list(images.resize(Images.Resolution.MEDIUM))
low_imgs = list(images.resize(Images.Resolution.LOW))
final_imgs = list(images.resize(Images.Resolution.FINAL))

plot_images(low_imgs, (20,20))

In [ ]:
original_size = images.sizes[0]
medium_size = images.get_image_size(medium_imgs[0])
low_size = images.get_image_size(low_imgs[0])
final_size = images.get_image_size(final_imgs[0])

print(f"Original Size: {original_size}  -> {'{:,}'.format(np.prod(original_size))} px ~ 1 MP")
print(f"Medium Size:   {medium_size}  -> {'{:,}'.format(np.prod(medium_size))} px ~ 0.6 MP")
print(f"Low Size:      {low_size}   -> {'{:,}'.format(np.prod(low_size))} px ~ 0.1 MP")
print(f"Final Size:    {final_size}  -> {'{:,}'.format(np.prod(final_size))} px ~ 1 MP")

## Find Features

For medium-sized images, our goal is to identify features that can characterize prominent elements present in the images. We will do this using the `FeatureDetector` class, with the intention of identifying similar features in other images.

In [ ]:
finder = FeatureDetector()
features = [finder.detect_features(img) for img in medium_imgs]
keypoints_center_img = finder.draw_keypoints(medium_imgs[1], features[1])
plot_image(keypoints_center_img, (15,10))

## Match Features

We can now compare the features of the paired images using the `FeatureMatcher` class. During this process, we examine confidences, computed as 

`confidence = number of inliers / (8 + 0.3 * number of matches)`

and inliers, determined through the (RANSAC) method.

In [ ]:
# From this, we can see which images have a low and high matching confidence with the other images. 
matcher = FeatureMatcher()
matches = matcher.match_features(features)
matcher.get_confidence_matrix(matches)

In [ ]:
all_relevant_matches = matcher.draw_matches_matrix(medium_imgs, features, matches, conf_thresh=1, 
                                                   inliers=True, matchColor=(0, 255, 0))
for idx1, idx2, img in all_relevant_matches:
    print(f"Matches Image {idx1+1} to Image {idx2+1}")
    plot_image(img, (20,10))

## Subset

As shown above, the noise image lacks any connection to the other images constituting the panorama. Our next step involves crafting a subset that includes only the pertinent images. To accomplish this, we employ the `Subsetter` class and define a `confidence_threshold` to determine when a match is considered a good match.

In [ ]:
subsetter = Subsetter()
dot_notation = subsetter.get_matches_graph(images.names, matches)
print(dot_notation)

These matches graph visualizes what we've saw in the confidence matrix. Now, we want to subset all variables.

In [ ]:
indices = subsetter.get_indices_to_keep(features, matches)
medium_imgs = subsetter.subset_list(medium_imgs, indices)
low_imgs = subsetter.subset_list(low_imgs, indices)
final_imgs = subsetter.subset_list(final_imgs, indices)
features = subsetter.subset_list(features, indices)
matches = subsetter.subset_matches(matches, indices)
images.subset(indices)
print(images.names)
print(matcher.get_confidence_matrix(matches))

## Camera Estimation, Adjustion and Correction

Our next objective involves calibrating cameras for the purpose of warping images, ensuring their correct composition. This process employs the `CameraEstimator`, `CameraAdjuster`, and `WaveCorrector` classes.

In [ ]:
camera_estimator = CameraEstimator()
camera_adjuster = CameraAdjuster()
wave_corrector = WaveCorrector()
cameras = camera_estimator.estimate(features, matches)
cameras = camera_adjuster.adjust(features, matches, cameras)
cameras = wave_corrector.correct(cameras)

## Warp Images

We now want to warp the images itself into the final plane, using the `Warper` class.

In [ ]:
warper = Warper()

# first set the medium focal length of the cameras as scale
warper.set_scale(cameras)

# warp low resolution images
low_sizes = images.get_scaled_img_sizes(Images.Resolution.LOW)
camera_aspect = images.get_ratio(Images.Resolution.MEDIUM, Images.Resolution.LOW)  
warped_low_imgs = list(warper.warp_images(low_imgs, cameras, camera_aspect))
warped_low_masks = list(warper.create_and_warp_masks(low_sizes, cameras, camera_aspect))
low_corners, low_sizes = warper.warp_rois(low_sizes, cameras, camera_aspect)

# warp final resolution images
final_sizes = images.get_scaled_img_sizes(Images.Resolution.FINAL)
camera_aspect = images.get_ratio(Images.Resolution.MEDIUM, Images.Resolution.FINAL)
warped_final_imgs = list(warper.warp_images(final_imgs, cameras, camera_aspect))
warped_final_masks = list(warper.create_and_warp_masks(final_sizes, cameras, camera_aspect))
final_corners, final_sizes = warper.warp_rois(final_sizes, cameras, camera_aspect)

# plot results
plot_images(warped_low_imgs, (10,10))
plot_images(warped_low_masks, (10,10))

In [ ]:
print(final_corners)
print(final_sizes)

## Crop

No individual image encompasses the entire height of the final plane. To generate a panorama devoid of black borders, we can proceed to calculate the largest common interior rectangle and subsequently crop the individual images accordingly, utilizing the functionalities of the `Cropper` class.

In [ ]:
cropper = Cropper()

# estimate panorama mask of potential final panorama using a Blender
mask = cropper.estimate_panorama_mask(warped_low_imgs, warped_low_masks, low_corners, low_sizes)
plot_image(mask, (5,5))

In [ ]:
lir = cropper.estimate_largest_interior_rectangle(mask)
print(lir)

In [ ]:
plot = lir.draw_on(mask, size=2)
plot_image(plot, (5,5))

In [ ]:
# by zero centering the warped corners, the rectangle of the images within the final plane can be determined
low_corners = cropper.get_zero_center_corners(low_corners)
rectangles = cropper.get_rectangles(low_corners, low_sizes)
plot = rectangles[1].draw_on(plot, (0, 255, 0), 2)  # The rectangle of the center img
plot_image(plot, (5,5))

In [ ]:
# using the overlap, the new corners and sizes can be determined 
overlap = cropper.get_overlap(rectangles[1], lir)
plot = overlap.draw_on(plot, (255, 0, 0), 2)
plot_image(plot, (5,5))

In [ ]:
# we are able to crop it now using the blue rectangle in the coordinate system of the original green image
intersection = cropper.get_intersection(rectangles[1], overlap)
plot = intersection.draw_on(warped_low_masks[1], (255, 0, 0), 2)
plot_image(plot, (2.5,2.5))

In [ ]:
# we can now crop the images and masks and obtain new corners and sizes using all this information 
cropper.prepare(warped_low_imgs, warped_low_masks, low_corners, low_sizes)
cropped_low_masks = list(cropper.crop_images(warped_low_masks))
cropped_low_imgs = list(cropper.crop_images(warped_low_imgs))
low_corners, low_sizes = cropper.crop_rois(low_corners, low_sizes)
lir_aspect = images.get_ratio(Images.Resolution.LOW, Images.Resolution.FINAL)  
cropped_final_masks = list(cropper.crop_images(warped_final_masks, lir_aspect))
cropped_final_imgs = list(cropper.crop_images(warped_final_imgs, lir_aspect))
final_corners, final_sizes = cropper.crop_rois(final_corners, final_sizes, lir_aspect)

## Seam Masks

Seam masks identify a transition line between images with minimal disruption, leveraging the capabilities of the `SeamFinder` class. The seams are derived from the warped low-resolution images and subsequently resized to match the resolution of the final warped images. These seam masks play a crucial role in the blending step, providing guidance on how the images should be seamlessly composed.

In [ ]:
seam_finder = SeamFinder()
seam_masks = seam_finder.find(cropped_low_imgs, low_corners, cropped_low_masks)
seam_masks = [seam_finder.resize(seam_mask, mask) for seam_mask, mask in zip(seam_masks, cropped_final_masks)]
seam_masks_plots = [SeamFinder.draw_seam_mask(img, seam_mask) for img, seam_mask in zip(cropped_final_imgs, seam_masks)]
plot_images(seam_masks_plots, (15,10))

## Exposure Error Compensation

Discrepancies in exposure errors across images can result in artifacts in the eventual panorama. These exposure errors are evaluated on the warped low-resolution images and subsequently applied to the warped final-resolution images, employing the functionalities of the `ExposureErrorCompensator` class.

In [ ]:
compensator = ExposureErrorCompensator()
compensator.feed(low_corners, cropped_low_imgs, cropped_low_masks)
compensated_imgs = [compensator.apply(idx, corner, img, mask) 
                    for idx, (img, mask, corner) 
                    in enumerate(zip(cropped_final_imgs, cropped_final_masks, final_corners))]

## Blending

The images can be seamlessly blended into a complete panorama through the utilization of the `Blender` class. The blend strength parameter determines the extent to which images should overlay along the transitions of the masks.

In [ ]:
blender = Blender()
blender.prepare(final_corners, final_sizes)
for img, mask, corner in zip(compensated_imgs, seam_masks, final_corners):
    blender.feed(img, mask, corner)
panorama, _ = blender.blend()
plot_image(panorama, (20,20))

We have the capability to illustrate the seams on the final panorama by plotting them as lines or polygons. This visualization allows us to discern which section of the panorama corresponds to each individual image. The approach involves blending single-colored dummy images with the acquired seam masks and adhering to the dimensions of the panorama.

In [ ]:
blended_seam_masks = seam_finder.blend_seam_masks(seam_masks, final_corners, final_sizes)
plot_image(blended_seam_masks, (5,5))

In [ ]:
# this blend can be converted into lines or weighted on top of resulting panorama
plot_image(seam_finder.draw_seam_lines(panorama, blended_seam_masks, linesize=3), (15,10))
plot_image(seam_finder.draw_seam_polygons(panorama, blended_seam_masks), (15,10))